In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from config import akey

In [2]:
file='api-data/yearly_summary/prices_for_'
company_file = 'api-data/company_info.csv'
years = ['2015','2016','2017','2018','2019','2020']
company = pd.read_csv(company_file)
#loop through years in list to open CSV files, group by Ticker, Industry and Sector. Then calculate Pct Change
#using the lamba function that pulls beg and end dates for each stock then calculates the change. 
#It then groups by sector to get a sector analysis for each year. 
for year in years:
    stocks = pd.read_csv(f'{file}{year}.csv')
    sector_perf = stocks.groupby(['Ticker','Sector','Industry'],sort=False).apply(lambda x: (x['Close'].values[-1] - x['Close'].values[0]) / x['Close'].values[-1] * 100)\
    .reset_index(name='pct change')
    sectors = sector_perf.groupby('Sector').mean()
    sectors.sort_values('pct change', ascending=False, inplace=True)
    sectors['Year'] = year
    sectors.to_csv(f'api-data/yearly_summary/sector_pct_chg_{year}.csv')
#same as above, but instead of looking at sectors we are looking at individual stock tickers
    ticker = stocks.groupby('Ticker',sort=False).apply(lambda x: (x['Close'].values[-1] - x['Close'].values[0]) / x['Close'].values[-1] * 100)\
    .reset_index(name='pct change')
    ticker.sort_values('pct change', ascending=False, inplace=True)
    tickers = sector_perf.merge(company, on='Ticker', how='left')
    tickers.drop(columns=['Industry_x','Sector_x'], inplace=True)
    tickers.rename(columns={'Industry_y':'Industry','Sector_y':'Sector'}, inplace=True)
    tickers['Year'] = year
    tickers.to_csv(f'api-data/yearly_summary/ticker_pct_chg_{year}.csv', index=False)